In [ ]:
import re

In [ ]:
workflows = dict()
parts = []

with open("day19_input.txt") as file:
    for line in file:
        if match := re.match(r"(\w+)\{(.+)\}", line):
            name = match.group(1)
            rules = match.group(2).split(",")
            workflows[name] = rules
        elif match := re.match(r"\{(.+)\}", line):
            parts.append(match.group(1))

In [ ]:
def eval_rules(rules, part):
    for category in part.split(","):
        exec(category)
    for rule in rules:
        *condition, result = rule.split(":")
        if (not condition) or eval(condition[0]):
            break
    return result

# Part 1


In [ ]:
total = 0
for part in parts:
    name = "in"
    while True:
        name = eval_rules(workflows[name], part)
        if name == "R":
            break
        elif name == "A":
            for number in re.findall(r"\d+", part):
                total += int(number)
            break

print("Answer:", total)

# Part 2
